In [20]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
from keras import preprocessing
import os
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.utils import to_categorical
from scipy import ndimage
from keras import models
from keras import layers
from keras.models import load_model
from keras.models import save_model
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
import tensorflow as tf
#from keras import legacy
from keras.preprocessing import image
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# Function to preprocess the dataset
def preprocess_dataset(data):
    data_resized = cv2.resize(data, (224, 224))    #resize
    data_normalized = data_resized / 255.0         #normalize
    return data_normalized


# the train and validation datasets
path = "/kaggle/input/nn-dataset/dataset/dataset/train"
def Train_data_preprocessing_(path):
    image_generator=ImageDataGenerator(
        rescale=1/255,               #Scales the pixel values to the range [0,1].
        rotation_range=10,           #Randomly rotates images by up to 10 degrees.
        width_shift_range=0.2,       #Randomly shifts images horizontally by up to 20% of the width.
        height_shift_range=0.2,      #Randomly shifts images vertically by up to 20% of the width.
        zoom_range=0.2,              # Randomly zooms into images by up to 20%.
        horizontal_flip=True,
        validation_split=0.2,)   #20% validation from the training set
    train_dataset=image_generator.flow_from_directory(batch_size=32,directory=path,
                                    shuffle=True,target_size=(224,224),subset="training",class_mode='categorical')
    test_dataset=image_generator.flow_from_directory(batch_size=32,directory=path,
                                  shuffle=True,target_size=(224,224),subset="validation",class_mode='categorical')
    return train_dataset,test_dataset 

train,test=Train_data_preprocessing_(path)

Found 7920 images belonging to 5 classes.
Found 1980 images belonging to 5 classes.


In [7]:
#2

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))  # convolutional layer 32filter with window size 3*3 
model.add(MaxPooling2D((2, 2)))           #reduces the spatial dimensions of the input data by maximum value from each block.
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten()) #This layer flattens the output from the previous layer into a 1D array. It prepares the data for the fully connected layers.
model.add(Dense(256, activation='relu')) #A fully connected layer with 256 units and a ReLU activation function.
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))  

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train,
    epochs=10, 
    validation_data=test
)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model2.h5', save_best_only=True)
# Training
model.fit(train, epochs=10, validation_data=test, callbacks=[early_stopping, model_checkpoint], verbose=1)

model.save('model2.h5')


Epoch 1/10
248/248 [==============================] - 138s 551ms/step - loss: 1.3875 - accuracy: 0.4130 - val_loss: 1.2437 - val_accuracy: 0.4722
Epoch 2/10
248/248 [==============================] - 137s 552ms/step - loss: 1.1219 - accuracy: 0.5327 - val_loss: 1.1822 - val_accuracy: 0.4960
Epoch 3/10
248/248 [==============================] - 137s 553ms/step - loss: 1.0332 - accuracy: 0.5818 - val_loss: 1.1149 - val_accuracy: 0.5394
Epoch 4/10
248/248 [==============================] - 137s 553ms/step - loss: 0.9840 - accuracy: 0.6044 - val_loss: 1.0628 - val_accuracy: 0.5591
Epoch 5/10
248/248 [==============================] - 137s 553ms/step - loss: 0.9380 - accuracy: 0.6280 - val_loss: 1.1186 - val_accuracy: 0.5444
Epoch 6/10
248/248 [==============================] - 138s 555ms/step - loss: 0.9171 - accuracy: 0.6391 - val_loss: 0.9841 - val_accuracy: 0.6081
Epoch 7/10
248/248 [==============================] - 140s 564ms/step - loss: 0.8860 - accuracy: 0.6549 - val_loss: 0.9881 -

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


248/248 [==============================] - 139s 560ms/step - loss: 0.8233 - accuracy: 0.6883 - val_loss: 0.9480 - val_accuracy: 0.6379
Epoch 2/10
248/248 [==============================] - 142s 572ms/step - loss: 0.8115 - accuracy: 0.6867 - val_loss: 0.9049 - val_accuracy: 0.6444
Epoch 3/10
248/248 [==============================] - 137s 553ms/step - loss: 0.7899 - accuracy: 0.7040 - val_loss: 0.9061 - val_accuracy: 0.6657
Epoch 4/10
248/248 [==============================] - 136s 549ms/step - loss: 0.7697 - accuracy: 0.7105 - val_loss: 0.9108 - val_accuracy: 0.6606
Epoch 5/10
248/248 [==============================] - 137s 554ms/step - loss: 0.7572 - accuracy: 0.7112 - val_loss: 0.9123 - val_accuracy: 0.6540


In [14]:
# Path to the test dataset
test_dataset_path = "/kaggle/input/nn-dataset/dataset/dataset/test"

def preprocess_test_dataset(test_dataset_path):
    preprocessed_data = []   # To store preprocessed images
    labels = []              # To store labels

    test_images_paths = [os.path.join(test_dataset_path, filename) for filename in os.listdir(test_dataset_path) if
                         filename.endswith(('.jpg', '.jpeg', '.png'))]   # Get paths of all images in the test dataset directory
     #read the images
    for image_path in test_images_paths:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Unable to read the image: {image_path}")
            continue

        # Preprocess the  image
        preprocessed_image = preprocess_dataset(image)
        preprocessed_data.append(preprocessed_image)

        # Extract label without file extension
        labels.append(os.path.splitext(os.path.basename(image_path))[0])
         # Convert the preprocessed data to a numpy array
        preprocessed_data = np.array(preprocessed_data)
        #shape handling
        if len(preprocessed_data.shape) == 3:
            preprocessed_data = preprocessed_data.reshape((-1, 224, 224, 3))

    return preprocessed_data, labels

# Modified test_images function
def test_images(test_dataset_path, model):
    preprocessed_data, labels = preprocess_test_dataset(test_dataset_path)

    if preprocessed_data.shape[0] == 0:
        print("No valid images in the test dataset.")
        return
    #make prediction
    predictions = model.predict(preprocessed_data)
    predicted_labels = np.argmax(predictions, axis=1) + 1

    results = pd.DataFrame({'image_id': labels, 'label': predicted_labels})
    print(results)

    csv_file_path = 'predictmodel2.csv'
    results.to_csv(csv_file_path, index=False)

#load the model
model2=load_model('model2.h5')
# Set the path to the test dataset
test_dataset_path = "/kaggle/input/nn-dataset/dataset/dataset/test"

# Test the images and save the results to a CSV file
test_images(test_dataset_path,model2)

4/4 [==============================] - 0s 76ms/step
   image_id  label
0      1266      3
1      1862      4
2       311      5
3      3362      2
4      3652      1
..      ...    ...
95     3813      1
96      639      5
97     3449      2
98     1656      2
99     1910      4

[100 rows x 2 columns]


In [52]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the Transformer model
def build_transformer_model(input_shape=(224, 224, 3), num_classes=5):
    inputs = layers.Input(shape=input_shape)

    # Preprocessing layers
    x = layers.Rescaling(scale=1.0 / 255)(inputs)     #scalling the input

    # Transformer layers
    backbone = tf.keras.applications.ResNet50(
        include_top=False,     #only the base convolutional layer
        weights='imagenet',  #from the model
        input_tensor=x,  #specifies the input tensor for the model
        pooling=None    #no global pooling will be applied by the ResNet50 model. The output will be the activation maps from the last convolutional layer.
    )
    x = backbone.output    #feature extractor from el backbone

    # Classification head with modifications
    x = layers.GlobalAveragePooling2D()(x)  #computes the spatial average of the entire feature map, reducing its spatial dimensions.
    x = layers.Dense(512, activation='relu')(x) 
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model with a learning rate schedule
    def lr_schedule(epoch, lr):         
        if epoch < 10:
            return lr
        else:
            return lr * tf.math.exp(-0.1)

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


    return model

# Modify the train_alexnet function to train the Transformer model
def train_transformer(model, train_data, test_data, epochs=100):
    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    # Define model checkpoint callback to save the best weights
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        'transformer_best.h5',
        save_best_only=True,
        save_weights_only=True,
        monitor='val_loss',
        mode='min',
        verbose=1
    )

    # Training
    model.fit(train_data, epochs=epochs, validation_data=test_data, callbacks=[early_stopping, model_checkpoint], verbose=1)

    return model

input_shape = (224, 224, 3)
num_classes = 5

# Build and train the Transformer model
transformer_model = build_transformer_model(input_shape, num_classes)
trained_model = train_transformer(transformer_model, train, test, epochs=20)
trained_model.save('transformer2.h5')

# Define the test_images function to test the Transformer model
def test_images(test_dataset_path, model):
    preprocessed_data, labels = preprocess_test_dataset(test_dataset_path)

    if preprocessed_data.shape[0] == 0:
        print("No valid images in the test dataset.")
        return

    predictions = model.predict(preprocessed_data)
    predicted_labels = np.argmax(predictions, axis=1) + 1

    results = pd.DataFrame({'image_id': labels, 'label': predicted_labels})
    print(results)

    csv_file_path = 'trans.csv'
    results.to_csv(csv_file_path, index=False)

# Load the trained model
transformer_model = tf.keras.models.load_model('transformer2.h5')

# Set the path to the test dataset
test_dataset_path = "/kaggle/input/nn-dataset/dataset/dataset/test"

# Test the images and save the results to a CSV file
test_images(test_dataset_path, transformer_model)

Epoch 1/20
248/248 [==============================] - ETA: 0s - loss: 2.0536 - accuracy: 0.2619
Epoch 1: val_loss improved from inf to 1.69748, saving model to transformer_best.h5
248/248 [==============================] - 186s 603ms/step - loss: 2.0536 - accuracy: 0.2619 - val_loss: 1.6975 - val_accuracy: 0.2000
Epoch 2/20
248/248 [==============================] - ETA: 0s - loss: 1.5147 - accuracy: 0.3524
Epoch 2: val_loss did not improve from 1.69748
248/248 [==============================] - 143s 574ms/step - loss: 1.5147 - accuracy: 0.3524 - val_loss: 1.8188 - val_accuracy: 0.2000
Epoch 3/20
248/248 [==============================] - ETA: 0s - loss: 1.3165 - accuracy: 0.4264
Epoch 3: val_loss did not improve from 1.69748
248/248 [==============================] - 142s 573ms/step - loss: 1.3165 - accuracy: 0.4264 - val_loss: 1.9218 - val_accuracy: 0.2000
Epoch 4/20
248/248 [==============================] - ETA: 0s - loss: 1.1310 - accuracy: 0.5449
Epoch 4: val_loss did not improve

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 1s 111ms/step
   image_id  label
0      1266      2
1      1862      2
2       311      2
3      3362      2
4      3652      2
..      ...    ...
95     3813      3
96      639      2
97     3449      2
98     1656      2
99     1910      2

[100 rows x 2 columns]


In [53]:
loss, accuracy = model.evaluate(test)

62/62 [==============================] - 27s 442ms/step - loss: 0.9160 - accuracy: 0.6399


In [22]:
#1
def build_alexnet(input_shape=(224, 224, 3), num_classes=5):
    model = Sequential()

    # Convolutional Block 1
    model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(BatchNormalization())

    # Convolutional Block 2
    model.add(Conv2D(256, (5, 5), padding='same', activation='relu'))
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(BatchNormalization())

    # Convolutional Block 3
    model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(BatchNormalization())

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='relu')) 
    model.add(Dense(num_classes, activation='softmax')) 

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def train_alexnet(model, train_data, test_data, epochs=100):
    # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) #if no change in loss after 10 epochs

    # Define model checkpoint callback to save the best weights
    model_checkpoint = ModelCheckpoint('alexnet_best22.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min', verbose=1)

    # Training
   
    model.fit(train_data, epochs=epochs, validation_data=test_data, callbacks=[early_stopping, model_checkpoint], verbose=1)
 

    return model

input_shape = (224, 224, 3)
num_classes = 5

# Build and train the model
alexnet_model = build_alexnet(input_shape, num_classes)
trained_model = train_alexnet(alexnet_model,train,test, epochs=20)
model=trained_model.save('model.h5')


# Path to the test dataset
test_dataset_path = "/kaggle/input/nn-dataset/dataset/dataset/test"

def preprocess_test_dataset(test_dataset_path):
    preprocessed_data = []
    labels = []

    test_images_paths = [os.path.join(test_dataset_path, filename) for filename in os.listdir(test_dataset_path) if
                         filename.endswith(('.jpg', '.jpeg', '.png'))]

    for image_path in test_images_paths:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Unable to read the image: {image_path}")
            continue

        # Preprocess the resized image
        preprocessed_image = preprocess_dataset(image)
        preprocessed_data.append(preprocessed_image)

        # Extract label without file extension
        labels.append(os.path.splitext(os.path.basename(image_path))[0])

    preprocessed_data = np.array(preprocessed_data)
    if len(preprocessed_data.shape) == 3:
        preprocessed_data = preprocessed_data.reshape((-1, 224, 224, 3))

    return preprocessed_data, labels

# Modified test_images function
def test_images(test_dataset_path, model):
    preprocessed_data, labels = preprocess_test_dataset(test_dataset_path)

    if preprocessed_data.shape[0] == 0:
        print("No valid images in the test dataset.")
        return
    #predictions according to our model
    predictions = model.predict(preprocessed_data)
    predicted_labels = np.argmax(predictions, axis=1) + 1

    results = pd.DataFrame({'image_id': labels, 'label': predicted_labels})
    print(results)

    csv_file_path = 'predictions.csv'
    results.to_csv(csv_file_path, index=False)

# Load the trained model
#trainedmodel = tf.keras.models.load_model('model.h5')
alex_model = build_alexnet() 
alex_model.load_weights('/kaggle/working/alexnet_best22.h5')
# Set the path to the test dataset
test_dataset_path = "/kaggle/input/nn-dataset/dataset/dataset/test"

# Test the images and save the results to a CSV file
test_images(test_dataset_path, alex_model)

Found 7920 images belonging to 5 classes.
Found 1980 images belonging to 5 classes.
Epoch 1/20
248/248 [==============================] - ETA: 0s - loss: 1.6554 - accuracy: 0.3963
Epoch 1: val_loss improved from inf to 1.98388, saving model to alexnet_best22.h5
248/248 [==============================] - 124s 485ms/step - loss: 1.6554 - accuracy: 0.3963 - val_loss: 1.9839 - val_accuracy: 0.2813
Epoch 2/20
248/248 [==============================] - ETA: 0s - loss: 1.2310 - accuracy: 0.4761
Epoch 2: val_loss improved from 1.98388 to 1.70369, saving model to alexnet_best22.h5
248/248 [==============================] - 121s 486ms/step - loss: 1.2310 - accuracy: 0.4761 - val_loss: 1.7037 - val_accuracy: 0.3071
Epoch 3/20
248/248 [==============================] - ETA: 0s - loss: 1.1480 - accuracy: 0.5279
Epoch 3: val_loss improved from 1.70369 to 1.26625, saving model to alexnet_best22.h5
248/248 [==============================] - 121s 486ms/step - loss: 1.1480 - accuracy: 0.5279 - val_loss:

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 1s 112ms/step
   image_id  label
0      1266      3
1      1862      4
2       311      2
3      3362      2
4      3652      3
..      ...    ...
95     3813      2
96      639      3
97     3449      2
98     1656      2
99     1910      4

[100 rows x 2 columns]
